In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate
import scipy.interpolate

In [2]:
# Set up a function for the comoving distance

def com(z_, OmC, OmB, HH0, Nnu):
    """ Gets the comoving distance in units of Mpc/h at a given redshift, z_ (assuming the cosmology defined in the params file). """
    
    OmR	=	2.47*10**(-5)/(HH0/100.)**2
    OmN	=	Nnu*(7./8.)*(4./11.)**(4./3.)*OmR
    OmL = 1. - OmC - OmB - OmR - OmN
    c=2.99792458*10**(8)
    H0	=	10**(5)/c
    
    def chi_int(z):
        return 1. / (H0 * ( (OmC+OmB)*(1+z)**3 + OmL + (OmR+OmN) * (1+z)**4 )**(0.5))

    if hasattr(z_, "__len__"):
        chi=np.zeros((len(z_)))
        for zi in range(0,len(z_)):
            #print "zi in com=", zi
            chi[zi] = scipy.integrate.quad(chi_int,0,z_[zi])[0]
    else:
        chi = scipy.integrate.quad(chi_int, 0, z_)[0]

    return chi

In [3]:
# Define a function to get redshift distribution (in terms of specz)
def get_NofZ(dNdzpar, dNdztype, z):
    """ Returns the dNdz of the sources."""

    if (dNdztype == 'Nakajima'):
        # dNdz takes form like in Nakajima et al. 2011 equation 3
        a = dNdzpar[0]
        zs = dNdzpar[1]
        nofz_ = (z / zs)**(a-1) * np.exp( -0.5 * (z / zs)**2)	
    elif (dNdztype == 'Smail'):
        # dNdz take form like in Smail et al. 1994
        alpha = dNdzpar[0]
        z0 = dNdzpar[1]
        beta = dNdzpar[2]
        nofz_ = z**alpha * np.exp( - (z / z0)**beta)
    else:
        print "dNdz type "+str(dNdztype)+" not yet supported; exiting."
        exit()
        
    norm = scipy.integrate.simps(nofz_, z)

    return nofz_ / norm

In [4]:
# Set up the Schechter luminosity function parameters from Krause et al. 2015

z = scipy.linspace(0.02, 5.00, 500)
# Define a few absolute magnitude limits for which we will check this:
mlim = 27

A_0 = 5.92; beta = 1.10; eta = -0.47; z_0 = 0.3
Mr_s = -20.34; Q = 1.8; alpha = -0.57
phi_0 = 1.1 * 10**(-2); P = -1.2

phi_s = phi_0 * 10**(0.4 * P*z)
Ms = Mr_s - Q * ( z - 0.1 )
Ls = 10.**(-0.4 *Ms)

In [6]:
# Import the k correction and EC correction from Poggianti (assuming elliptical galaxies)
# Only data up to z=3 so have just extended at a constant the highest-z value
(z_k, kcorr, x,x,x) = np.loadtxt('../txtfiles/kcorr.dat', unpack=True)
(z_e, ecorr, x,x,x) = np.loadtxt('../txtfiles/ecorr.dat', unpack=True)

kcorr_interp = scipy.interpolate.interp1d(z_k, kcorr)
ecorr_interp = scipy.interpolate.interp1d(z_e, ecorr)

kcorr = kcorr_interp(z)
ecorr = ecorr_interp(z)

In [7]:
# Get the absolute magnitude that corresponds to the limiting apparently magnitude
dl = com(z, 0.2, 0.05, 70., 3.046) * (1. + z)

Mlim = mlim - (5. * np.log10(dl) + 25. + kcorr + ecorr)
Llim = 10.**(-0.4 * Mlim)

In [8]:
L_list = [0]*len(z)
phi_func = np.zeros((len(z), 1000))
for i in range(0,len(z)):
    for j in range(0,1000):
        #print "i=", i, "j=", j
        L = scipy.logspace(np.log10(Llim[i]), 10., 1000)
        L_list[i] = L
        phi_func[i,j] = phi_s[i] * (L[j] / Ls[i]) ** alpha * np.exp(- L[j] / Ls[i])
        

In [9]:
L0 = 10.**(-0.4 * (-22.))

A = [0]*(len(z))
for i in range(0, len(z)):
    num = scipy.integrate.simps(phi_func[i, :] * (L_list[i] / L0)**beta, L_list[i])
    denom = scipy.integrate.simps(phi_func[i, :] , L_list[i])
    A[i] = A_0 * ((1. + z[i]) / (1. + z_0))**eta * num/ denom

In [10]:
# Get the normalized redshift distribution in terms of spec z
dNdz = get_NofZ([1.24, 0.51, 1.01], 'Smail', z)

# and integrate over it

A_intz = scipy.integrate.simps(A*dNdz, z)
print "A=", A_intz

A= 4.28229096098
